# Plan-and-execute


Papers: Plan-and-execute agents accomplish an objective by first planning what to do, then executing the sub tasks. This idea is largely inspired by [BabyAGI](https://github.com/yoheinakajima/babyagi) and then the ["Plan-and-Solve" paper](https://arxiv.org/abs/2305.04091).

The planning is almost always done by an LLM.

The execution is usually done by a separate agent (equipped with tools).

<img src="./img/planner.png" width="50%" height=50%>

In [ ]:
%pip install -U openai
%pip install -U langchain
%pip install -U langchain_community
%pip install -U langchain_core
%pip install -U langchain_experimental
%pip install -U langchain_openai
%pip install -U duckduckgo-search
%pip install -U numexpr

In [1]:
from langchain.chains import LLMMathChain
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain_core.tools import Tool
from langchain_experimental.plan_and_execute import (
    PlanAndExecute,
    load_agent_executor,
    load_chat_planner,
)
from langchain_openai import ChatOpenAI, OpenAI

## Tools

In [2]:
search = DuckDuckGoSearchAPIWrapper()
llm = OpenAI(temperature=0.4)
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math",
    ),
]

In [ ]:
llm = ChatOpenAI(model="gpt-3.5", temperature=0.5)
# class that wraps another class and logs all function calls being executed 
class Wrapper:
    def __init__(self, wrapped_class):
        self.wrapped_class = wrapped_class

    def __getattr__(self, attr):
        original_func = getattr(self.wrapped_class, attr)

        def wrapper(*args, **kwargs):
            print(f"Calling function: {attr}")
            print(f"Arguments: {args}, {kwargs}")
            result = original_func(*args, **kwargs)
            print(f"Response: {result}")
            return result

        return wrapper

# overwrite the private `client` attribute inside of the LLM that contains the API client with our wrapped class
llm.client = Wrapper(llm.client)

## Planner, Executor, and Agent


In [3]:
model = ChatOpenAI(temperature=0)
planner = load_chat_planner(model)
executor = load_agent_executor(model, tools, verbose=True)
agent = PlanAndExecute(planner=planner, executor=executor)

## Run example

In [11]:
#agent.invoke("Who is the current prime minister of the UK? What is their current age raised to the 0.43 power?")
agent.invoke({"input":"Who is the current prime minister of the UK? What is their current age raised to the 0.43 power?"})



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Search",
  "action_input": {
    "type": "string"
  }
}
```
Observation: A string is a sequence of characters, terminated by a null character, in contiguous memory locations. Strings have ordered, indexable, and comparable properties, and are widely used in data structures and algorithms. Learn about the characteristics, applications, advantages, disadvantages, and examples of strings. A string is an object of type String whose value is text. Learn how to declare, initialze, manipulate, and compare strings in C# with the string keyword, the String class, and the StringInfo object. See examples of quoted, verbatim, and raw string literals. What is String? Strings are defined as an array of characters. The difference between a character array and a string is the string is terminated with a special character '\0'. String Data Structure. Below are some examples of strings: "geeks", "for", "geeks", "GeeksforGeeks", "Geeks fo

ToolException: Too many arguments to single-input tool Calculator. Args: ['calculate', 'current_age ^ 0.43']